In [ ]:
import struct
import os
import numpy as np
import pandas as pd
import random
import plotly.graph_objects as go
import math
import sys
import datetime
import re

In [ ]:
#
# Pvalues calculations for PXD000561, REMOVING the experiment files without identifications
#

PVALUES_MIXEDSPECTRACRUX_FULL_01="/media/eduseiti/data_storage_1TB/unicamp/PXD000561/mixedSpectraCrux_experiments_removing_files_without_identifications_pvalues/300.dat_allSpectraPvalues.bin"
PVALUES_MIXEDSPECTRACRUX_FULL_02="/media/eduseiti/data_storage_1TB/unicamp/PXD000561/mixedSpectraCrux_experiments_removing_files_without_identifications_pvalues/474.dat_allSpectraPvalues.bin"
PVALUES_MIXEDSPECTRACRUX_FULL_03="/media/eduseiti/data_storage_1TB/unicamp/PXD000561/mixedSpectraCrux_experiments_removing_files_without_identifications_pvalues/560.dat_allSpectraPvalues.bin"
PVALUES_MIXEDSPECTRACRUX_FULL_04="/media/eduseiti/data_storage_1TB/unicamp/PXD000561/mixedSpectraCrux_experiments_removing_files_without_identifications_pvalues/632.dat_allSpectraPvalues.bin"
PVALUES_MIXEDSPECTRACRUX_FULL_05="/media/eduseiti/data_storage_1TB/unicamp/PXD000561/mixedSpectraCrux_experiments_removing_files_without_identifications_pvalues/715.dat_allSpectraPvalues.bin"
PVALUES_MIXEDSPECTRACRUX_FULL_06="/media/eduseiti/data_storage_1TB/unicamp/PXD000561/mixedSpectraCrux_experiments_removing_files_without_identifications_pvalues/818.dat_allSpectraPvalues.bin"
PVALUES_MIXEDSPECTRACRUX_FULL_07="/media/eduseiti/data_storage_1TB/unicamp/PXD000561/mixedSpectraCrux_experiments_removing_files_without_identifications_pvalues/1005.dat_allSpectraPvalues.bin"

#
# Can use the same .tsv file, since the access is done through the .mgf filename.
#

COUNTS_MIXEDSPECTRACRUX_FULL="/media/eduseiti/bigdata01/unicamp/doutorado/clustering/linfeng/sample/pvalue/PXD000561_all_experiments_spectra_count.tsv"
LIST_OF_FILES_FOR_PVALUES="/media/eduseiti/data_storage_1TB/unicamp/PXD000561/PXD000561_all_experiments_removing_files_without_identifications.txt"



#
# Initial pvalues calculations for PXD000561, WITHOUT removing the experiment files without identifications
#

# PVALUES_MIXEDSPECTRACRUX_FULL_01="/media/eduseiti/bigdata01/unicamp/doutorado/clustering/linfeng/sample/pvalue/PXD000561_all_experiments_pvalues/300.dat_allSpectraPvalues.bin"
# PVALUES_MIXEDSPECTRACRUX_FULL_02="/media/eduseiti/bigdata01/unicamp/doutorado/clustering/linfeng/sample/pvalue/PXD000561_all_experiments_pvalues/474.dat_allSpectraPvalues.bin"
# PVALUES_MIXEDSPECTRACRUX_FULL_03="/media/eduseiti/bigdata01/unicamp/doutorado/clustering/linfeng/sample/pvalue/PXD000561_all_experiments_pvalues/560.dat_allSpectraPvalues.bin"
# PVALUES_MIXEDSPECTRACRUX_FULL_04="/media/eduseiti/bigdata01/unicamp/doutorado/clustering/linfeng/sample/pvalue/PXD000561_all_experiments_pvalues/632.dat_allSpectraPvalues.bin"
# PVALUES_MIXEDSPECTRACRUX_FULL_05="/media/eduseiti/bigdata01/unicamp/doutorado/clustering/linfeng/sample/pvalue/PXD000561_all_experiments_pvalues/715.dat_allSpectraPvalues.bin"
# PVALUES_MIXEDSPECTRACRUX_FULL_06="/media/eduseiti/bigdata01/unicamp/doutorado/clustering/linfeng/sample/pvalue/PXD000561_all_experiments_pvalues/818.dat_allSpectraPvalues.bin"
# PVALUES_MIXEDSPECTRACRUX_FULL_07="/media/eduseiti/bigdata01/unicamp/doutorado/clustering/linfeng/sample/pvalue/PXD000561_all_experiments_pvalues/1005.dat_allSpectraPvalues.bin"

# COUNTS_MIXEDSPECTRACRUX_FULL="/media/eduseiti/bigdata01/unicamp/doutorado/clustering/linfeng/sample/pvalue/PXD000561_all_experiments_spectra_count.tsv"


#
# Original experiments from PXD000561, selected for the first tests of "mixedSpectraCrux"
#

# PVALUES_MIXEDSPECTRACRUX="/mnt/f633ac7c-3153-4566-a009-229a0ae5f8a1/unicamp/doutorado/clustering/linfeng/sample/mixedSpectraCrux_experiments_pvalues/300.dat_allSpectraPvalues.bin"
# COUNTS_MIXEDSPECTRACRUX="/media/eduseiti/ebee9cb2-c63c-48bf-b862-004aba7612ee/unicamp/mixedSpectraCrux/mixedSpectraCrux_experiment_spectra_count.tsv"


#
# Initial test data - Lifeng 10% sample
#

# PVALUES_FILE="/mnt/f633ac7c-3153-4566-a009-229a0ae5f8a1/unicamp/doutorado/clustering/linfeng/sample/test_pvalues_fix/400.dat_allSpectraPvalues.bin"
# IDS_FILE="/mnt/f633ac7c-3153-4566-a009-229a0ae5f8a1/unicamp/doutorado/clustering/linfeng/sample/identifications_sample_0.1_nterm/sample_experiment_identifications/percolator.target.psms.txt"
# COUNTS_FILE="/mnt/f633ac7c-3153-4566-a009-229a0ae5f8a1/unicamp/doutorado/spectra_count.tsv"

### Define the identifications minimum score

In [ ]:
# EXPERIMENTS_MGF_DEFINITIONS_FOLDER="/media/eduseiti/data_storage_500MB/unicamp/PXD000561"

# sys.path.insert(1, EXPERIMENTS_MGF_DEFINITIONS_FOLDER)

# import original_experiments_mgf_definitions
import all_experiments_mgf_definitions as definitions

In [ ]:
# For q < 0.01

# all_identifications_to_use = definitions.IDENTIFICATIONS

# For q < 0.001

all_identifications_to_use = definitions.IDENTIFICATIONS_Q_001

The "all_experiments_mgf_definitions" holds data structures used by the PXD000561.py code in the model to travel across the data structures of that work experiments

In [ ]:
len(all_identifications_to_use)

### The code below is just a verification of the existing files; does not need to be executed

This dataframe contains the spectra count for each .mgf file, in the same sequence as given to the maracluster to calculate the pvalues.

In [ ]:
countsdf = pd.read_csv(COUNTS_MIXEDSPECTRACRUX_FULL, sep="\t")

print(countsdf)

print(countsdf['spectra count'].sum())

countsa = np.array(countsdf)

print(countsa[0][0])

Now, verify if the number of files listed for each experiment matches the whole total for that experiment

In [ ]:
i = 0

with open("/media/eduseiti/data_storage_500MB/unicamp/PXD000561/PXD000561_all_experiments.txt", "r") as inputFile:
    
    structure_name = ""
    last_index = 0
    how_many = 0
    
    while True:
        line = inputFile.readline()

        if not line:
            break
            
        hasFile = re.match(".+\/(.+)\_[0-9]{1,2}\_f([0-9]{2})\.mgf$", line)
        
        if not hasFile:
            print("Could not match: peaks file={}, mgf file={}".format(countsa[i][0], line))
        else:
            if hasFile.group(1) != structure_name:
                if len(structure_name) > 0:
                    if how_many < last_index:
                        print("This one is missing: {}, counted={} files, last index={}".format(structure_name, how_many, last_index))
                    else:
                        print("Correct: {}, counted={} files, last index={}".format(structure_name, how_many, last_index))
                    
                    how_many = 0
                    
                structure_name = hasFile.group(1)
                how_many += 1
            else:
                how_many += 1
                last_index = int(hasFile.group(2))
            
        i += 1

In [ ]:
# del sys.modules['all_experiments_mgf_definitions']
# del definitions

## Starting here is the code to merge the maracluster pvalues with the dataset identification files

In [ ]:
OUTPUT_FOLDER="/media/eduseiti/data_storage_1TB/unicamp/PXD000561/all_experiments_q0001_filtered_identifications_original_with_pvalues"

In [ ]:
STRUCT_FIELDS = "IId"
NO_PVALUE = sys.maxsize

### This function will decode the binary file created by maracluster with each spectra pvalue

The structure will contain the pvalues calculated for each spectra inside the .mgf file passed as parameter, in the order given by the files list given to maracluster as input.

In [ ]:
def decode_pvalues_file(pvalues_filename):
    
    pvalues = []

    with open(pvalues_filename, "rb") as inputFile:
        while True:
            record = inputFile.read(struct.calcsize(STRUCT_FIELDS))

            if not record:
                break
            else:
                unpacked = struct.unpack_from(STRUCT_FIELDS, record)
                
                pvalues.append(unpacked)
                
                if math.isnan(unpacked[2]):
                    print("nan: {}".format(record))

    print("Decoded {} pvalues from {}".format(len(pvalues), pvalues_filename))
    
    return np.array(pvalues)

In [ ]:
def plot_pvalues_histogram(pvalues):
    
    pvalues_df = pd.DataFrame(pvalues, columns = ["file", "scannr", "pvalue"])
    
    print(pvalues_df['pvalue'].describe(percentiles=list(np.round(np.arange(0.0, 1.0, 0.05), 2))))
    
    pvalues_histogram, pvalues_bin_edges = np.histogram(pvalues_df['pvalue'], 1000)

    fig = go.Figure()

    fig.add_trace(go.Bar(y=pvalues_histogram,
                         x=pvalues_bin_edges[1:],
                         marker_color='red'))
    
    fig.show()
    
    return pvalues_df, pvalues_histogram, pvalues_bin_edges

In [ ]:
pvalues_01 = decode_pvalues_file(PVALUES_MIXEDSPECTRACRUX_FULL_01)
pvalues_02 = decode_pvalues_file(PVALUES_MIXEDSPECTRACRUX_FULL_02)
pvalues_03 = decode_pvalues_file(PVALUES_MIXEDSPECTRACRUX_FULL_03)
pvalues_04 = decode_pvalues_file(PVALUES_MIXEDSPECTRACRUX_FULL_04)
pvalues_05 = decode_pvalues_file(PVALUES_MIXEDSPECTRACRUX_FULL_05)
pvalues_06 = decode_pvalues_file(PVALUES_MIXEDSPECTRACRUX_FULL_06)
pvalues_07 = decode_pvalues_file(PVALUES_MIXEDSPECTRACRUX_FULL_07)

In [ ]:
pvalues = np.concatenate((pvalues_01, pvalues_02, pvalues_03, pvalues_04, pvalues_05, pvalues_06, pvalues_07))

In [ ]:
pvalues_df, _, _ = plot_pvalues_histogram(pvalues)

In [ ]:
pvalues_df.columns

In [ ]:
pvalues_df = pvalues_df.sort_values(['file', 'scannr'])

In [ ]:
pvalues_df

In [ ]:
len(all_identifications_to_use)

### Make a validation of the pvalues dataframe

In [ ]:
pvalues_array = pvalues_df.to_numpy()

FILE_IDX = pvalues_df.columns.get_loc("file")
SCANNR_IDX = pvalues_df.columns.get_loc("scannr")
PVALUE_IDX = pvalues_df.columns.get_loc("pvalue")

current_file = -1
first_scan = -1
last_scan = -1
num_of_pvalues = 0

validation_result = []

for i in range(len(pvalues_array)):
    if pvalues_array[i][FILE_IDX] != current_file:
        if current_file != -1:
            new_result = {}
            new_result['file idx'] = current_file
            new_result['first scan'] = first_scan
            new_result['last scan'] = last_scan
            new_result['num of pvalues'] = num_of_pvalues
            
            validation_result.append(new_result)
            
        current_file = pvalues_array[i][FILE_IDX]
        first_scan = pvalues_array[i][SCANNR_IDX]
        num_of_pvalues = 1
    else:
        last_scan = pvalues_array[i][SCANNR_IDX]
        num_of_pvalues += 1

#
# Append the last file result
#

new_result = {}
new_result['file idx'] = current_file
new_result['first scan'] = first_scan
new_result['last scan'] = last_scan
new_result['num of pvalues'] = num_of_pvalues

validation_result.append(new_result)
        

In [ ]:
validation_result_df = pd.DataFrame(validation_result)

In [ ]:
validation_result_df

In [ ]:
list_of_pvalue_files_df = pd.read_csv(LIST_OF_FILES_FOR_PVALUES, names=["file"], sep="\t")

In [ ]:
list_of_pvalue_files_df['file'] = list_of_pvalue_files_df['file'].str.split("/").str[-1]

In [ ]:
validation_result_df['file'] = list_of_pvalue_files_df['file']

In [ ]:
validation_result_df

In [ ]:
validation_final_df = pd.merge(countsdf, validation_result_df, on='file', how='outer')

In [ ]:
validation_final_df.shape

In [ ]:
validation_final_df.to_csv(os.path.join(OUTPUT_FOLDER, "pvalues_validation.tsv"), sep='\t', index=False)

### Read the pvalues calculated in maracluster and set then in the corresponding experiment identification .tsv file

This function will go through the list of experiments' identification files.

For each file, it will go through the identifications and get the corresponding pvalue from the pvalues array created by maracluster, matching the FILE INDEX and the SCAN INDEX.

In [ ]:
pvalues_idx = 0

total_ignored_pvalues_first = 0

current_experiment_first_general_file_idx = 0
current_experiment_total_files = 0

pvalues_array = pvalues_df.to_numpy()

FILE_IDX = pvalues_df.columns.get_loc("file")
SCANNR_IDX = pvalues_df.columns.get_loc("scannr")
PVALUE_IDX = pvalues_df.columns.get_loc("pvalue")

final_report = []

pvalues_identifications = []

#
# Loop through the identifications .tsv files of each experiment
#

for file in all_identifications_to_use:
        
    # Advance in pvalues file until finishing the previous experiment file scans
    
    while pvalues_array[pvalues_idx][FILE_IDX] < current_experiment_first_general_file_idx + current_experiment_total_files:
#         print("Advancing at the start. pvalues_array[pvalues_idx][FILE_IDX]={}, current_experiment_first_general_file_idx={}, current_experiment_total_files={}".format(pvalues_array[pvalues_idx][FILE_IDX], 
#                                                                                                                                                                         current_experiment_first_general_file_idx, 
#                                                                                                                                                                         current_experiment_total_files))
        pvalues_idx += 1
        
        total_ignored_pvalues_first += 1

    if pvalues_array[pvalues_idx][FILE_IDX] > current_experiment_first_general_file_idx + current_experiment_total_files:
        print("Missing pvalues for first experiment mgf file. pvalues_array[pvalues_idx][FILE_IDX]={}, current_experiment_first_general_file_idx={}, current_experiment_total_files={}".format(pvalues_array[pvalues_idx][FILE_IDX], 
                                                                                                                                                                        current_experiment_first_general_file_idx, 
                                                                                                                                                                        current_experiment_total_files))
        raise ValueError()
        
        
        
    # Update the general file index to start the current experiment
        
    current_experiment_first_general_file_idx += current_experiment_total_files
        
        
    # Read the current experiment data
        
    identifications_df = pd.read_csv(file, sep="\t")
    
    identifications_array = identifications_df.to_numpy()

    ID_FILE_IDX = identifications_df.columns.get_loc("file_idx")
    ID_SCAN = identifications_df.columns.get_loc("scan")
    
    
    filename = file.split('/')[-1]
        
    current_experiment_total_files = len(definitions.MATCHES_TO_CRUX_FILES_LIST[filename])  
    
    identifications_pvalues = []
    
    experiment_identifications_without_pvalue = 0
    processed_experiment_identifications = 0

    print("\n\nHandling experiment={}, with {} files and a with total of {} identifications".format(filename, 
                                                                                                  current_experiment_total_files, 
                                                                                                  identifications_df.shape[0]))
       

    last_processed_experiment_file = -1
    how_many_identifications_without_pvalue = 0
    
    total_identifications_experiment_file = 0
    identifications_without_pvalue_file = 0

    new_results = {}
        
    #
    # Go through all the identification scans for the given file
    #
        
    for identification in identifications_array:
        
        experiment_general_file_idx = current_experiment_first_general_file_idx + identification[ID_FILE_IDX]

        if last_processed_experiment_file != identification[ID_FILE_IDX]:
            
            if len(new_results) > 0:
                new_results['total identifications'] = total_identifications_experiment_file
                new_results['identifications with pvalue'] = total_identifications_experiment_file - identifications_without_pvalue_file
                new_results['identifications without pvalue'] = identifications_without_pvalue_file
                
                final_report.append(new_results)
                
                total_identifications_experiment_file = 0
                identifications_without_pvalue_file = 0                

                
            new_results = {}
            
            new_results['file'] = definitions.MATCHES_TO_CRUX_FILES_LIST[filename][identification[ID_FILE_IDX]]
            
            
            print("\n\n---- Filling pvalues for experiment file index={}, general file index={}. Starting pvalue_idx={}".format(identification[ID_FILE_IDX], 
                                                                                                                         experiment_general_file_idx, 
                                                                                                                         pvalues_idx))
            last_processed_experiment_file = identification[ID_FILE_IDX]
            
            
        processed_experiment_identifications += 1
        total_identifications_experiment_file += 1
        
        # Advance pvalues file until reaching the experiment general file index
        
        while pvalues_array[pvalues_idx][FILE_IDX] < experiment_general_file_idx and pvalues_idx < len(pvalues_array) - 1:
            pvalues_idx += 1
        
#         print("-- Handling experiment scan={}, pvalues index={}".format(identification[ID_SCAN], pvalues_idx))

        found_identification_scan = False

        # Advance in pvalues file until reaching the identification scan

        while (pvalues_array[pvalues_idx][SCANNR_IDX] < identification[ID_SCAN]) and \
              (pvalues_array[pvalues_idx][FILE_IDX] == experiment_general_file_idx) and pvalues_idx < len(pvalues_array) - 1:
            pvalues_idx += 1

        if pvalues_array[pvalues_idx][FILE_IDX] == experiment_general_file_idx:
            if pvalues_array[pvalues_idx][SCANNR_IDX] == identification[ID_SCAN]:
                identifications_pvalues.append(pvalues_array[pvalues_idx][PVALUE_IDX])

#                 print("--- Found scan in pvalues index={}: {}".format(pvalues_idx, pvalues_array[pvalues_idx]))


                pvalues_identifications.append({"file": experiment_general_file_idx, 
                                                "scan": identification[ID_SCAN], 
                                                "pvalue": pvalues_array[pvalues_idx][PVALUE_IDX]})

                pvalues_idx += 1

                found_identification_scan = True


        if not found_identification_scan:
            identifications_pvalues.append(NO_PVALUE)

#             print("------ Identification scan={} had no pvalue".format(identification[ID_SCAN]))
            how_many_identifications_without_pvalue += 1
            identifications_without_pvalue_file += 1


    # Save report information of the last experiment file
    
    if len(new_results) > 0:
        new_results['total identifications'] = total_identifications_experiment_file
        new_results['identifications with pvalue'] = total_identifications_experiment_file - identifications_without_pvalue_file
        new_results['identifications without pvalue'] = identifications_without_pvalue_file

        final_report.append(new_results)    
    
    identifications_df['pvalue'] = identifications_pvalues

    identifications_df.to_csv(os.path.join(OUTPUT_FOLDER, file.split('/')[-1]), index=False, sep='\t')

    print("-- FINISHED after processing {}/{} identifications, with {} identifications scans without pvalues.".format(processed_experiment_identifications, 
                                                                                                                      identifications_df.shape[0],
                                                                                                                      how_many_identifications_without_pvalue))

final_report_df = pd.DataFrame(final_report)

datetime_execution = str(datetime.datetime.now())

final_report_df.to_csv(os.path.join(OUTPUT_FOLDER, "spectra_pvalues_report_" + datetime_execution + ".tsv"), index=False, sep='\t')
   
pvalues_identifications_df = pd.DataFrame(pvalues_identifications)
pvalues_identifications_df.to_csv(os.path.join(OUTPUT_FOLDER, "pvalues_identifications" + datetime_execution + ".tsv"), index=False, sep='\t')

In [ ]:
total_ignored_pvalues_first

In [ ]:
print(pvalues_identifications_df['pvalue'].describe(percentiles=list(np.round(np.arange(0.0, 1.0, 0.05), 2))))

pvalues_identifications_histogram, pvalues_identifications_bin_edges = np.histogram(pvalues_identifications_df['pvalue'], 1000)

fig = go.Figure()

fig.add_trace(go.Bar(y=pvalues_identifications_histogram,
                     x=pvalues_identifications_bin_edges[1:],
                     marker_color='red'))

fig.show()

In [ ]:
pvalues_df['pvalue'].describe(percentiles=list([0.1, 0.01, 0.001]))

In [ ]:
pvalues_identifications_df['pvalue'].describe(percentiles=list([0.1, 0.01, 0.001]))

### Create a report with the statistics for each individual file

In [ ]:
count_df = pd.read_csv(COUNTS_MIXEDSPECTRACRUX_FULL, sep='\t')

In [ ]:
final_report_df

In [ ]:
final_report_totals_df = count_df.set_index('file').join(final_report_df.set_index('file'))

In [ ]:
final_report_totals_df.to_csv(os.path.join(OUTPUT_FOLDER, "final_report_with_totals" + datetime_execution + ".tsv"), index=False, sep='\t')

In [ ]:
final_report_totals_df.sum()